In [6]:
import numpy as np
import tensorflow as tf
from IPython.display import clear_output, Image, display, HTML


In [2]:
# creating fresh Graph and TensorFlow session
graph = tf.Graph()
sess = tf.InteractiveSession(graph=graph)

In [3]:
# Prepare input for the format expected by the graph
t_input = tf.placeholder(np.float32, name='our_input') # define the input tensor
imagenet_mean = 117.0
t_preprocessed = tf.expand_dims(t_input-imagenet_mean, 0)

# Load graph and import into graph used by our session
model_fn = '/home/ubuntu/tensorflow_inception_graph.pb'
graph_def = tf.GraphDef.FromString(open(model_fn).read())
tf.import_graph_def(graph_def, {'input':t_preprocessed})

In [16]:
layers = [op.name for op in graph.get_operations() if op.type=='Conv2D' and 'import/' in op.name]
feature_nums = [int(graph.get_tensor_by_name(name+':0').get_shape()[-1]) for name in layers]

In [19]:
print(layers)

[u'import/conv2d0_pre_relu/conv', u'import/conv2d1_pre_relu/conv', u'import/conv2d2_pre_relu/conv', u'import/mixed3a_1x1_pre_relu/conv', u'import/mixed3a_3x3_bottleneck_pre_relu/conv', u'import/mixed3a_3x3_pre_relu/conv', u'import/mixed3a_5x5_bottleneck_pre_relu/conv', u'import/mixed3a_5x5_pre_relu/conv', u'import/mixed3a_pool_reduce_pre_relu/conv', u'import/mixed3b_1x1_pre_relu/conv', u'import/mixed3b_3x3_bottleneck_pre_relu/conv', u'import/mixed3b_3x3_pre_relu/conv', u'import/mixed3b_5x5_bottleneck_pre_relu/conv', u'import/mixed3b_5x5_pre_relu/conv', u'import/mixed3b_pool_reduce_pre_relu/conv', u'import/mixed4a_1x1_pre_relu/conv', u'import/mixed4a_3x3_bottleneck_pre_relu/conv', u'import/mixed4a_3x3_pre_relu/conv', u'import/mixed4a_5x5_bottleneck_pre_relu/conv', u'import/mixed4a_5x5_pre_relu/conv', u'import/mixed4a_pool_reduce_pre_relu/conv', u'import/mixed4b_1x1_pre_relu/conv', u'import/mixed4b_3x3_bottleneck_pre_relu/conv', u'import/mixed4b_3x3_pre_relu/conv', u'import/mixed4b_5x5_b

In [22]:
type(layers)

list

In [23]:
for layer in layers:
    print(layer)

import/conv2d0_pre_relu/conv
import/conv2d1_pre_relu/conv
import/conv2d2_pre_relu/conv
import/mixed3a_1x1_pre_relu/conv
import/mixed3a_3x3_bottleneck_pre_relu/conv
import/mixed3a_3x3_pre_relu/conv
import/mixed3a_5x5_bottleneck_pre_relu/conv
import/mixed3a_5x5_pre_relu/conv
import/mixed3a_pool_reduce_pre_relu/conv
import/mixed3b_1x1_pre_relu/conv
import/mixed3b_3x3_bottleneck_pre_relu/conv
import/mixed3b_3x3_pre_relu/conv
import/mixed3b_5x5_bottleneck_pre_relu/conv
import/mixed3b_5x5_pre_relu/conv
import/mixed3b_pool_reduce_pre_relu/conv
import/mixed4a_1x1_pre_relu/conv
import/mixed4a_3x3_bottleneck_pre_relu/conv
import/mixed4a_3x3_pre_relu/conv
import/mixed4a_5x5_bottleneck_pre_relu/conv
import/mixed4a_5x5_pre_relu/conv
import/mixed4a_pool_reduce_pre_relu/conv
import/mixed4b_1x1_pre_relu/conv
import/mixed4b_3x3_bottleneck_pre_relu/conv
import/mixed4b_3x3_pre_relu/conv
import/mixed4b_5x5_bottleneck_pre_relu/conv
import/mixed4b_5x5_pre_relu/conv
import/mixed4b_pool_reduce_pre_relu/conv
i

In [29]:
print(len(layers))
print(len(feature_nums))
print(feature_nums)

59
59
[64, 64, 192, 64, 96, 128, 16, 32, 32, 128, 128, 192, 32, 96, 64, 192, 96, 204, 16, 48, 64, 160, 112, 224, 24, 64, 64, 128, 128, 256, 24, 64, 64, 112, 144, 288, 32, 64, 64, 256, 160, 320, 32, 128, 128, 256, 160, 320, 48, 128, 128, 384, 192, 384, 48, 128, 128, 128, 128]


In [46]:
#printing layers and features
i=0
for layer in layers:
    print  "Layer[" + str(i+1) + "]: " + layer + " - Total Features : (%d)"  %feature_nums[i]
    i=i+1


Layer[1]: import/conv2d0_pre_relu/conv - Total Features : (64)
Layer[2]: import/conv2d1_pre_relu/conv - Total Features : (64)
Layer[3]: import/conv2d2_pre_relu/conv - Total Features : (192)
Layer[4]: import/mixed3a_1x1_pre_relu/conv - Total Features : (64)
Layer[5]: import/mixed3a_3x3_bottleneck_pre_relu/conv - Total Features : (96)
Layer[6]: import/mixed3a_3x3_pre_relu/conv - Total Features : (128)
Layer[7]: import/mixed3a_5x5_bottleneck_pre_relu/conv - Total Features : (16)
Layer[8]: import/mixed3a_5x5_pre_relu/conv - Total Features : (32)
Layer[9]: import/mixed3a_pool_reduce_pre_relu/conv - Total Features : (32)
Layer[10]: import/mixed3b_1x1_pre_relu/conv - Total Features : (128)
Layer[11]: import/mixed3b_3x3_bottleneck_pre_relu/conv - Total Features : (128)
Layer[12]: import/mixed3b_3x3_pre_relu/conv - Total Features : (192)
Layer[13]: import/mixed3b_5x5_bottleneck_pre_relu/conv - Total Features : (32)
Layer[14]: import/mixed3b_5x5_pre_relu/conv - Total Features : (96)
Layer[15]: i

In [11]:
print 'Number of layers', len(layers)
print 'Total number of feature channels:', sum(feature_nums)

Number of layers 59
Total number of feature channels: 7548


In [7]:
# Helper functions for TF Graph visualization
def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def
  
def rename_nodes(graph_def, rename_func):
    res_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = res_def.node.add() 
        n.MergeFrom(n0)
        n.name = rename_func(n.name)
        for i, s in enumerate(n.input):
            n.input[i] = rename_func(s) if s[0]!='^' else '^'+rename_func(s[1:])
    return res_def
  
def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))
  
    iframe = """
        <iframe seamless style="width:800px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

# Visualizing the network graph. Be sure expand the "mixed" nodes to see their 
# internal structure. We are going to visualize "Conv2D" nodes.
tmp_def = rename_nodes(graph_def, lambda s:"/".join(s.split('_',1)))
show_graph(tmp_def)